# Scenario Builder
*Use this notebook to generate new scenarios for the simulation.*

In [ ]:
%%capture
%matplotlib inline

import GWorld
import Agent
import PlotGWorld
plotgw = PlotGWorld.PlotGWorld(); # Object for accessing plotters
import pprint
import json
import numpy as np
rng = np.random.default_rng(seed=0)

The Scenario Buiilder creates new scenarios using already existing scenarios as a template.

## Inputs
- `Scenario_source_file` : Name of the JSON file which has the **template scenario**
- `Scenario_destination_file` : Name of the JSON file to which the **new scenario** is to be saved. 
- `Template_Scenario` : Name of the **template scenario** in `Scenario_source_file` which is to be used as the template for the **new scenario**
- `new_scenario_name` : Name of the **new scenario**

In [ ]:
# For creating a New Scenarion based on Some Scenario
Template_Scenario = 'GameMap'
new_scenario_name = 'GameMap_Walls'

Scenario_source_file = 'Scenarios4FeARSims.json'
Scenario_destination_file = 'Scenarios4FeARSims.json'

Scenario = GWorld.LoadJsonScenario(json_filename=Scenario_source_file,scenario_name=Template_Scenario)

# Converting back to JSON format
Scenario= json.loads(json.dumps(Scenario,indent=1, sort_keys=True))

# View Template Scenario

In [ ]:
# View  Template Scenario
pretty_print_json = pprint.pformat(Scenario,width=150).replace("'", '"')
print(pretty_print_json)

---


# Generate New Map
The following commands can be useful in defining a NewMap for the scenario

- The Map of the GWorld Scenario is represented by a Matrix of ones and zeros. 
- **Ones** represent **valid locations** the agents in the simulation can occupy.
- **Zeros** represent **invalid locations which cannot be occupied by agents.

In [ ]:
# # NewMap = np.ones([5,10])
NewMap = np.zeros([10,16])
# NewMap = np.ones([10,20])

marginx = 4
marginy = 1

NewMap[0,:] = 1
NewMap[-1,:] = 1
NewMap[:,0] = 1
NewMap[:,-1] = 1


NewMap[:,marginx+1] = 1
NewMap[:,-(marginx+2)] = 1

NewMap[marginy+1,marginx+1:-(marginx+2)] = 1
NewMap[-(marginy+2),marginx+1:-(marginx+2)] = 1


# # NewMap[:,9] = 1
# # NewMap[:,4] = 1
# # NewMap[:,5] = 1
# # NewMap[:,2] = 1
# NewMap[:,7] = 1
# NewMap[:,8] = 1


print(NewMap)

# Make changes to the values of the Template

- `Overwrite = False` ensures that already existing scenarios are not overwritten.

---

- Use **DirectionWeights** and **StepWeights** to select the relative preference for action selection. These together define the action selection policy of an agent.
- **DirectionWeights** and **StepWeights** are the default values for all agents.
- To specify different values for specific agents use **SpecificDirectionWeights4Agents** and **SpecificStepWeights4Agents**

---

- **OneWays** allow for movement in one direction only (from start to stop).
- Actions that cause movements from the stop to start of a OneWay results is invalid and trigger collisions
- Define OneWays as list of tuples of adjacent locations [ [[x_start,y_start], [x_stop,y_stop]], [[x_start,y_start], [x_stop,y_stop]], ... ]

---

- **Walls** are also defined between adjacent cells. But movement is blocked in both directions.
- Define Walls as list of tuples of adjacent locations [ [[x_start,y_start], [x_stop,y_stop]], [[x_start,y_start], [x_stop,y_stop]], ... ]

---

In [ ]:
# Making Changes

# Overwrite = False
Overwrite = True


Scenario['Map']['OneWays'] = [ [[2,5],[2,6]], [[7,10],[7,9]], [[2,10],[3,10]], [[7,5],[6,5]]  ]
Scenario['Map']['Walls'] = [ [[1,5],[2,5]], [[7,10],[8,10]] ]
Scenario['Map']['Region'] = NewMap.tolist()

# 'DirectionWeights' = [Up,Down,left,Right]
# Scenario['DirectionWeights'] = [0,0,0,1]
# Scenario['StepWeights'] = [1,1,1,1,1]
# Scenario['AgentLocations'] = [[2,3],[2,5],[1,7]]
Scenario['N_Agents'] = 4
Scenario['N_iterations'] = 10
Scenario['N_Cases'] = 1
# Scenario['SpecificStepWeights4Agents'] = []
# Scenario['SpecificDirectionWeights4Agents'] = [[[2],[0,1,0,0]]]


# Review New Scenario

In [ ]:
# Pretty Formating

pretty_print_json = pprint.pformat(Scenario,width=150).replace("'", '"')
print(pretty_print_json)

# Preview New Scenario

In [ ]:
# GWorld Preview

Region = np.array(Scenario['Map']['Region'])
Walls = Scenario['Map']['Walls']
OneWays = Scenario['Map']['OneWays']

World = GWorld.GWorld(Region, Walls= Walls, OneWays = OneWays) # Initialising GWorld from Scenario

N_Agents = Scenario['N_Agents']
AgentLocations = Scenario['AgentLocations'].copy()
    
    
AgentLocations = []
for location in Scenario['AgentLocations']:
    AgentLocations.append(tuple(location))

# Adding N Agents at sorted random positions
if len(AgentLocations) < N_Agents:
    [locX,locY] = np.where(Region==1)

    LocIdxs = rng.choice(locX.shape[0], size=(N_Agents-len(AgentLocations)), replace=False, shuffle=False)
    LocIdxs.sort()

    for Idx in LocIdxs:
        AgentLocations.append((locX[Idx],locY[Idx]))

# Adding Agents
PreviousAgentAdded = True
for location in AgentLocations:
    # Adding new Agents if Previous Agent was Added to the World
    if PreviousAgentAdded: 
        Ag_i = Agent.Agent()
    PreviousAgentAdded = World.AddAgent(Ag_i,location, printStatus=False)

PreviousAgentAdded = True
while len(World.AgentList) < N_Agents:
    # Adding new Agents if Previous Agent was Added to the World
    if PreviousAgentAdded: 
        Ag_i = Agent.Agent()
    Loc_i = (np.random.randint(Region.shape[0]),np.random.randint(Region.shape[1]))
    PreviousAgentAdded = World.AddAgent(Ag_i,Loc_i, printStatus=False)


# Plotting the State of the World and Chosen Actions for the next iteration
plotgw.ViewGWorld(World,ViewActionArrows=False ,ViewActionTrail=False);

In [ ]:
# Add Scenario to JSON file

GWorld.AddJsonScenario(json_filename=Scenario_destination_file,new_scenario=Scenario, new_scenario_name=new_scenario_name, Overwrite = Overwrite)